# Fit the parameter recovery period for national data

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import differential_evolution
from datetime import datetime

from sirds_model import get_bounds_and_arguments, sirds_objective_function

## Reading data

In [2]:
df = pd.read_csv('data/output/df_ts_epidemic_episodes.csv', index_col=0)
df.DATA = pd.to_datetime(df.DATA)

## Preparing data

In [3]:
df = df.sort_values(by=['DATA'])

In [4]:
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'] = df['CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL']/df['POPULACAO_2022'] * 100000 

In [5]:
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'] = df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'].fillna(0)

In [6]:
df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'] = df['TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL'].replace([np.inf, -np.inf], 0)

## Executing model

In [ ]:
try:
    df_initial_results = pd.read_csv('data/output/fitting_recovery_period_results.csv')
except:
    df_initial_results = pd.DataFrame({'days_to_recovery':[]})

df_results = df_initial_results.copy()

DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR = 5
NUMBER_ESTIMATON_PER_RECOVERY_PERIOD = 20

# df = df[:90]

date_first_case = min(df[(df.NOVOS_CASOS_SRAG > 0)].iloc[0].DATA, df[(df.CASOS_NOVOS_PAINEL > 0)].iloc[0].DATA)
max_date_to_fit = df.DATA.max()

# Period of analysis
period_in_days = (max_date_to_fit - date_first_case).days + 1

for estimation in range(NUMBER_ESTIMATON_PER_RECOVERY_PERIOD):
    
    # Considering recovery period of 14 days for mild cases as reported by “Report of the WHO-China Joint Mission on Coronavirus Disease 2019 (COVID-19).” Accessed September 26, 2022. https://www.who.int/publications-detail-redirect/report-of-the-who-chinajoint-mission-on-coronavirus-disease-2019-(covid-19).
    # Considering recovery period mean between 13 and 15 second Voinsky, Irena, Gabriele Baristaite, and David Gurwitz. "Effects of age and sex on recovery from COVID-19: Analysis of 5769 Israeli patients." Journal of Infection 81.2 (2020): e102-e103.
    # Also using 14 days as mean +/- 6 SD we have range 8-20 or 8-26 if we considerate incubation period
    for days_to_recovery in range(8,21):
        print('days_to_recovery: '+str(days_to_recovery))
                   
        estimations_performed = len(df_results[(df_results.days_to_recovery == days_to_recovery)])
    
        if (estimation == estimations_performed):
            print('estimation: '+str(estimation))
            
            # Record the start time
            start_time = datetime.now()
            print(start_time)
    
            bounds, args = get_bounds_and_arguments(df, 'DATA', 'TAXA_OBITOS_NOVOS_MEDIA_MOVEL_7_DIAS_SIM', 'NUMERO_REPRODUCAO_EFETIVO_SRAG_MEDIA', 'TAXA_CASOS_NOVOS_MEDIA_MOVEL_7_DIAS_PAINEL', 'ONSET_NUMERO_REPRODUCAO_EFETIVO_MEDIA', days_to_recovery, date_first_case, max_date_to_fit, df.POPULACAO_2022.iloc[0], period_in_days)

            result = differential_evolution(sirds_objective_function, bounds, args=args, popsize=DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR, maxiter=10000, workers=4, updating='deferred')
    
            # Record the end time
            end_time = datetime.now()
    
            # Calculate the duration (in seconds) for the optimization
            duration = (end_time - start_time).total_seconds()
            print(duration)

            list_breakpoints_in_slow_transition = args[4]
            quantity_outbreaks = args[5]
            quantity_outbreak_adjustments = args[6]
    
            # Create a dictionary to store results
            estimation_result = {
                'estimation': estimation,  # To differentiate between multiple estimations
                'result_fun': result.fun,
                'result_nfev': result.nfev,
                'result_nit': result.nit,
                'result_success': result.success,
                'start_time': start_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format start time as a string
                'end_time': end_time.strftime('%Y-%m-%d %H:%M:%S'),  # Format end time as a string
                'duration_seconds': duration,  # Duration in seconds
                'pop_size': DIFFERENTIAL_EVOLUTION_POP_SIZE_FACTOR,
                'period_in_days': period_in_days,
                'days_to_recovery': days_to_recovery,
                'date_first_case': date_first_case,
                'list_breakpoints_in_slow_transition': list_breakpoints_in_slow_transition,
                'x_initial_infected_population': result.x[0],
                'x_days_between_infections_0': result.x[1],            
            }
            
            quantity_epidemic_periods_with_slow_transition = len(list_breakpoints_in_slow_transition) + 1            
            
            for p in range(quantity_epidemic_periods_with_slow_transition):
                estimation_result['x_case_fatality_probability_'+str(p)] = result.x[2 + p]
                estimation_result['x_loss_immunity_in_days_'+str(p)] = result.x[2 + quantity_epidemic_periods_with_slow_transition + p]                                                 
            quantity_breakpoints = (quantity_outbreaks - 1) + (quantity_outbreak_adjustments)
            begin_breakpoint_parameters = 2 + 2*quantity_epidemic_periods_with_slow_transition
            for b in range(quantity_breakpoints):
                estimation_result['x_days_between_infections_'+str(b+1)] = result.x[begin_breakpoint_parameters + b]
                estimation_result['x_breakpoint_'+str(b+1)] = result.x[begin_breakpoint_parameters + quantity_breakpoints + b]
                estimation_result['x_transition_days_between_epidemic_periods_'+str(b+1)] = result.x[begin_breakpoint_parameters + 2*quantity_breakpoints + b]
    
            print(estimation_result)
    
            # Append the estimation result to the list            
            df_results = pd.concat([df_results, pd.DataFrame.from_records([estimation_result])])
    
            df_results.to_csv('data/output/fitting_recovery_period_results.csv', index=False)           

days_to_recovery: 8
days_to_recovery: 9
days_to_recovery: 10
days_to_recovery: 11
days_to_recovery: 12
days_to_recovery: 13
days_to_recovery: 14
days_to_recovery: 15
days_to_recovery: 16
days_to_recovery: 17
days_to_recovery: 18
days_to_recovery: 19
days_to_recovery: 20
days_to_recovery: 8
days_to_recovery: 9
days_to_recovery: 10
days_to_recovery: 11
days_to_recovery: 12
days_to_recovery: 13
days_to_recovery: 14
days_to_recovery: 15
days_to_recovery: 16
days_to_recovery: 17
days_to_recovery: 18
days_to_recovery: 19
days_to_recovery: 20
days_to_recovery: 8
days_to_recovery: 9
days_to_recovery: 10
days_to_recovery: 11
days_to_recovery: 12
days_to_recovery: 13
days_to_recovery: 14
days_to_recovery: 15
days_to_recovery: 16
days_to_recovery: 17
days_to_recovery: 18
days_to_recovery: 19
days_to_recovery: 20
days_to_recovery: 8
days_to_recovery: 9
days_to_recovery: 10
days_to_recovery: 11
days_to_recovery: 12
days_to_recovery: 13
days_to_recovery: 14
days_to_recovery: 15
days_to_recovery: 16


In [ ]:
bounds